# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
import s3fs
import boto3
import pandas as pd
import nltk

import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import tensorflow as tf
import transformers
import numpy as np


2022-07-27 12:33:40.058717: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-27 12:33:40.063159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 12:33:40.063173: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import torch

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [32]:
dataset = pd.read_csv('BERTopic_Labeled.csv')

In [33]:
dataset['tokens']

0      ['wall', 'street', 'journal', 'amazon', 'bests...
1      ['helping', 'set', 'stage', 'biowares', 'hotly...
2      ['sebastian', 'locke', 'fiftysixyearold', 'pat...
3      ['take', 'back', 'oxmoon', 'lost', 'paradise',...
4      ['mayflower', 'set', 'sail', 'carried', 'board...
                             ...                        
995    ['lee', 'wants', 'tarantula', 'member', 'bigge...
996    ['merry', 'adventures', 'robin', 'hood', 'grea...
997    ['moving', 'presentday', 'oslo', 'brooklyn', '...
998    ['captivating', 'sequel', 'inkheart', 'critica...
999    ['santa', 'claus', 'dear', 'old', 'friend', 't...
Name: tokens, Length: 1000, dtype: object

In [34]:
def make_string(text):
    text_preprocessed = (" ").join(text)
    return text_preprocessed

In [35]:
from ast import literal_eval
dataset['tokens'] = dataset['tokens'].apply(lambda row: literal_eval(row))
dataset['tokens'] = dataset['tokens'].apply(lambda x: make_string(x))

In [36]:
dataset.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)

In [37]:
dataset = dataset[['ID','description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship']]

In [38]:
dataset

,ID,description,university,relationships,break ups,divorce,weddings,death,family,friendship
0,"39,822",From the Wall Street Journal and #1 Amazon bes...,False,True,False,False,False,False,False,False
1,"34,235",Helping set the stage for BioWare's hotly anti...,False,False,False,False,False,False,False,False
2,"27,904","Sebastian Locke, the fifty-six-year-old patria...",False,True,False,True,False,True,True,False
3,"10,515","""Take me back to Oxmoon, the lost paradise of ...",False,True,False,False,False,False,True,False
4,935,"When the Mayflower set sail in 1620, it carrie...",False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
995,"17,361",Lee wants to be a Tarantula – a member of the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
996,"9,029",The Merry Adventures of Robin Hood of Great Re...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,"32,216",Moving from present-day Oslo to Brooklyn in th...,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
998,"1,036","The captivating sequel to INKHEART, the critic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
dataset = dataset.replace(np.nan, False)

In [40]:
dataset[0:699].to_csv('dataset_bert_train.csv')
dataset[700:899].to_csv('dataset_bert_test.csv')
dataset[900:999].to_csv('dataset_bert_validation.csv')
dataset[700:999].to_csv('dataset_bert_test_val.csv')

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [41]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': ['dataset_bert_train.csv'], 'test': 'dataset_bert_test.csv', 'validation':'dataset_bert_validation.csv'})

Using custom data configuration default-2e8fb01ee019b551


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/csv/default-2e8fb01ee019b551/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Let's check the first example of the training split:

In [42]:
example = dataset['train'][0]
example

{'Unnamed: 0': 0,
 'ID': '39,822',
 'description': 'From the Wall Street Journal and #1 Amazon bestselling author comes a new installment to the Blue Moon small town romance series.  A womanizing bad boy with a motorcycle and sexy-as-sin smile is not part of Emma’s life plan.  She moved cross-country to be close to family and finally settle down in hippie, trippy, nosey Blue Moon Bend. But when famed fashion photographer Niko shows up with his leather jacket, underwear-melting voice, and a problem, she sees nothing but trouble.  He’s all wrong for Emma, but that doesn’t stop the attraction from boiling over.  Niko doesn’t let being friend zoned get in his way. Once he gets his hands—and his mouth—on her, will their friendship survive? Or will he lose everything he’s worked for back in New York to the redhead who dominates his every thought? ',
 'university': False,
 'relationships': True,
 'break ups': False,
 'divorce': False,
 'weddings': False,
 'death': False,
 'family': False,
 'f

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [43]:
labels = [label for label in dataset['train'][0].keys() if label not in ['ID', 'description', 'Unnamed: 0']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['university',
 'relationships',
 'break ups',
 'divorce',
 'weddings',
 'death',
 'family',
 'friendship']

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ID', 'description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'],
        num_rows: 699
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ID', 'description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'],
        num_rows: 199
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'ID', 'description', 'university', 'relationships', 'break ups', 'divorce', 'weddings', 'death', 'family', 'friendship'],
        num_rows: 99
    })
})

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [45]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples['description']
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/ec2-user/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/r

In [46]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [47]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [48]:
tokenizer.decode(example['input_ids'])

'[CLS] from the wall street journal and # 1 amazon bestselling author comes a new installment to the blue moon small town romance series. a womanizing bad boy with a motorcycle and sexy - as - sin smile is not part of emma ’ s life plan. she moved cross - country to be close to family and finally settle down in hippie, trippy, nosey blue moon bend. but when famed fashion photographer niko shows up with his leather jacket, underwear - melting voice, and a problem, she sees nothing but trouble. he ’ s all wrong for emma, but that doesn ’ t stop the attraction from boiling over [SEP]'

In [49]:
example['labels']

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [50]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['relationships']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [51]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [52]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/ec2-user/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "university",
    "1": "relationships",
    "2": "break ups",
    "3": "divorce",
    "4": "weddings",
    "5": "death",
    "6": "family",
    "7": "friendship"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "break ups": 2,
    "death": 5,
    "divorce": 3,
    "family": 6,
    "friendship": 7,
    "relationships": 1,
    "university": 0,
    "weddings": 4
 

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [53]:
batch_size = 8
metric_name = "f1"

In [54]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [55]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [56]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [57]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2013,  1996,  2813,  2395,  3485,  1998,  1001,  1015,  9733,
         2190, 23836,  2075,  3166,  3310,  1037,  2047, 18932,  2000,  1996,
         2630,  4231,  2235,  2237,  7472,  2186,  1012,  1037,  2450,  6026,
         2919,  2879,  2007,  1037,  9055,  1998,  7916,  1011,  2004,  1011,
         8254,  2868,  2003,  2025,  2112,  1997,  5616,  1521,  1055,  2166,
         2933,  1012,  2016,  2333,  2892,  1011,  2406,  2000,  2022,  2485,
         2000,  2155,  1998,  2633,  7392,  2091,  1999,  5099, 14756,  1010,
         4440,  7685,  1010,  4451,  2100,  2630,  4231,  8815,  1012,  2021,
         2043, 15607,  4827,  8088, 23205,  2080,  3065,  2039,  2007,  2010,
         5898,  6598,  1010, 14236,  1011, 13721,  2376,  1010,  1998,  1037,
         3291,  1010,  2016,  5927,  2498,  2021,  4390,  1012,  2002,  1521,
         1055,  2035,  3308,  2005,  5616,  1010,  2021,  2008,  2987,  1521,
         1056,  2644,  1996,  8432,  2013, 16018,  2058,   102])

In [58]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7572, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.3458,  0.4416,  0.7730, -0.0211, -0.0524,  0.1227,  0.4445, -0.4809]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [59]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [60]:
trainer.train()

/home/ec2-user/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 699
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 440


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.259024,0.000000,0.500000,0.363636
2,No log,0.232916,0.418182,0.635733,0.505051
3,No log,0.214242,0.512000,0.685719,0.545455
4,No log,0.218540,0.403670,0.629709,0.515152
5,No log,0.208285,0.457627,0.657009,0.515152


***** Running Evaluation *****
  Num examples = 99
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-88
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-88/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-88/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-88/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-88/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 99
  Batch size = 8
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-176
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-176/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-176/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-176/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-176/speci

TrainOutput(global_step=440, training_loss=0.27062329378995026, metrics={'train_runtime': 1449.7065, 'train_samples_per_second': 2.411, 'train_steps_per_second': 0.304, 'total_flos': 229905669335040.0, 'train_loss': 0.27062329378995026, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [61]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 99
  Batch size = 8


{'eval_loss': 0.21424183249473572,
 'eval_f1': 0.512,
 'eval_roc_auc': 0.6857188981596343,
 'eval_accuracy': 0.5454545454545454,
 'eval_runtime': 10.1389,
 'eval_samples_per_second': 9.764,
 'eval_steps_per_second': 1.282,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

In [63]:
BFT_test_val_results = pd.read_csv('dataset_bert_test_val.csv')
BFT_test_val_results['description'] = BFT_test_val_results['description'].apply(lambda x: re.sub(r'[^a-zA-Z ]+', '', x))

In [68]:
BFT_test_val_results['description'][293]

'When tragedy strikes Michele Windsors family she is forced to move from Los Angeles to New York City to live with the wealthy aristocratic grandparents she has never met In their historic Fifth Avenue mansion filled with a centurys worth of family secrets Michele discovers the biggest family secret of all  an ancestors diary that amazingly has the power to send her back in time to  the year it was written There at a glamorous highsociety masquerade ball Michele meets the young man with striking blue eyes who has haunted her dreams all her life And she finds herself falling for him and into an otherworldly romanceSoon Michele is leading a double life struggling to balance her contemporary high school world with her escapes into the past But when she stumbles upon a terrible discovery she is propelled on a race through history to save the boy she loves  and to complete a quest that will determine their fate'

In [69]:
test_probabilities = []
for i in range(0,len(BFT_test_val_results)):
    text = BFT_test_val_results['description'][i]
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    outputs = trainer.model(**encoding)
    logits = outputs.logits
    print(i, logits.shape)
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    test_probabilities.append(probs.tolist())
    #predictions = np.zeros(probs.shape)
    #predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
    #predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

0 torch.Size([1, 8])
1 torch.Size([1, 8])
2 torch.Size([1, 8])
3 torch.Size([1, 8])
4 torch.Size([1, 8])
5 torch.Size([1, 8])
6 torch.Size([1, 8])
7 torch.Size([1, 8])
8 torch.Size([1, 8])
9 torch.Size([1, 8])
10 torch.Size([1, 8])
11 torch.Size([1, 8])
12 torch.Size([1, 8])
13 torch.Size([1, 8])
14 torch.Size([1, 8])
15 torch.Size([1, 8])
16 torch.Size([1, 8])
17 torch.Size([1, 8])
18 torch.Size([1, 8])
19 torch.Size([1, 8])
20 torch.Size([1, 8])
21 torch.Size([1, 8])
22 torch.Size([1, 8])
23 torch.Size([1, 8])
24 torch.Size([1, 8])
25 torch.Size([1, 8])
26 torch.Size([1, 8])
27 torch.Size([1, 8])
28 torch.Size([1, 8])
29 torch.Size([1, 8])
30 torch.Size([1, 8])
31 torch.Size([1, 8])
32 torch.Size([1, 8])
33 torch.Size([1, 8])
34 torch.Size([1, 8])
35 torch.Size([1, 8])
36 torch.Size([1, 8])
37 torch.Size([1, 8])
38 torch.Size([1, 8])
39 torch.Size([1, 8])
40 torch.Size([1, 8])
41 torch.Size([1, 8])
42 torch.Size([1, 8])
43 torch.Size([1, 8])
44 torch.Size([1, 8])
45 torch.Size([1, 8]

In [70]:
BFT_test_val_results['scores'] = test_probabilities

In [71]:
BFT_test_val_results.to_csv('bert_finetuned_test_val_results_2.csv')

In [51]:
#text = "I'm happy I can finally train a model for multi-label classification"
text = dataset['test'][0]['description']
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [52]:
logits = outputs.logits
logits.shape

torch.Size([1, 8])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [58]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
print(probs.tolist())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

[0.04387282207608223, 0.8390726447105408, 0.04064767807722092, 0.03451037406921387, 0.06476005911827087, 0.07106984406709671, 0.18297196924686432, 0.25362786650657654]
['relationships']


In [ ]:
['university',
 'relationships',
 'break ups',
 'divorce',
 'weddings',
 'death',
 'family',
 'friendship']